In [1]:
!pip install arcgis --quiet
!pip install ipython --quiet
!pip install ipywidgets --quiet!pip install openpyxl --quiet

## Setup supporting packages

In [2]:
from IPython.display import display, HTML;

from arcgis.gis import GIS;
from arcgis.features import FeatureLayer,FeatureSet;
from arcgis.geocoding import geocode;

import requests,json;

## Load all HUC12s from 1408

In [3]:
huc4 = '1408' # Upper San Juan

wbd_fl = FeatureLayer(
    'https://watersgeo.epa.gov/arcgis/rest/services/Support/HydrologicUnits/MapServer/6'
);

wbd_rez = wbd_fl.query(
    where = "SUBSTRING(HUC12,1,4) = '" + huc4 + "'"
   ,out_fields = 'HUC12,NAME'
   ,return_geometry = False
   ,orderByFields = 'HUC12'
);

wbd_rez.sdf.head()

,HUC12,NAME,OBJECTID
0,140801010101,Headwaters East Fork of the San Juan River,37600
1,140801010102,Quartz Creek,37669
2,140801010103,Sand Creek,37670
3,140801010104,The Clam Shell-East Fork San Juan River,37599
4,140801010201,Beaver Creek,36640


## Query ATTAINS by HUC12

In [4]:
wbd_df1 = wbd_rez.sdf.drop(columns="OBJECTID");
wbd_df1 = wbd_df1.rename(columns={"NAME": "Name"});

def pull_attains(huc12):
    response = requests.get(
        'https://attains.epa.gov/attains-public/api/huc12summary?huc=' + huc12
    );
    strip = response.json();
    
    return strip['items'][0];

wbd_df1['ATTAINS_RESULTS'] = wbd_df1['HUC12'].apply(pull_attains);

wbd_df1.head()

,HUC12,Name,ATTAINS_RESULTS
0,140801010101,Headwaters East Fork of the San Juan River,"{'huc12': '140801010101', 'assessmentUnitCount..."
1,140801010102,Quartz Creek,"{'huc12': '140801010102', 'assessmentUnitCount..."
2,140801010103,Sand Creek,"{'huc12': '140801010103', 'assessmentUnitCount..."
3,140801010104,The Clam Shell-East Fork San Juan River,"{'huc12': '140801010104', 'assessmentUnitCount..."
4,140801010201,Beaver Creek,"{'huc12': '140801010201', 'assessmentUnitCount..."


## Skim off the HUC12 header data

In [5]:
wbd_df2 = wbd_df1;

wbd_df2['assessmentUnitCount']                       = wbd_df2['ATTAINS_RESULTS'].apply(lambda x: x['assessmentUnitCount']);
wbd_df2['totalCatchmentAreaSqMi']                    = wbd_df2['ATTAINS_RESULTS'].apply(lambda x: x['totalCatchmentAreaSqMi']);
wbd_df2['assessedCatchmentAreaSqMi']                 = wbd_df2['ATTAINS_RESULTS'].apply(lambda x: x['assessedCatchmentAreaSqMi']);
wbd_df2['assessedCatchmentAreaPercent']              = wbd_df2['ATTAINS_RESULTS'].apply(lambda x: x['assessedCatchmentAreaPercent']);
wbd_df2['assessedGoodCatchmentAreaSqMi']             = wbd_df2['ATTAINS_RESULTS'].apply(lambda x: x['assessedGoodCatchmentAreaSqMi']);
wbd_df2['assessedGoodCatchmentAreaPercent']          = wbd_df2['ATTAINS_RESULTS'].apply(lambda x: x['assessedGoodCatchmentAreaPercent']);
wbd_df2['assessedUnknownCatchmentAreaSqMi']          = wbd_df2['ATTAINS_RESULTS'].apply(lambda x: x['assessedUnknownCatchmentAreaSqMi']);
wbd_df2['assessedUnknownCatchmentAreaPercent']       = wbd_df2['ATTAINS_RESULTS'].apply(lambda x: x['assessedUnknownCatchmentAreaPercent']);
wbd_df2['containImpairedWatersCatchmentAreaSqMi']    = wbd_df2['ATTAINS_RESULTS'].apply(lambda x: x['containImpairedWatersCatchmentAreaSqMi']);
wbd_df2['containImpairedWatersCatchmentAreaPercent'] = wbd_df2['ATTAINS_RESULTS'].apply(lambda x: x['containImpairedWatersCatchmentAreaPercent']);
wbd_df2['containRestorationCatchmentAreaSqMi']       = wbd_df2['ATTAINS_RESULTS'].apply(lambda x: x['containRestorationCatchmentAreaSqMi']);
wbd_df2['containRestorationCatchmentAreaPercent']    = wbd_df2['ATTAINS_RESULTS'].apply(lambda x: x['containRestorationCatchmentAreaPercent']);

wbd_df2.head()

,HUC12,Name,ATTAINS_RESULTS,assessmentUnitCount,totalCatchmentAreaSqMi,assessedCatchmentAreaSqMi,assessedCatchmentAreaPercent,assessedGoodCatchmentAreaSqMi,assessedGoodCatchmentAreaPercent,assessedUnknownCatchmentAreaSqMi,assessedUnknownCatchmentAreaPercent,containImpairedWatersCatchmentAreaSqMi,containImpairedWatersCatchmentAreaPercent,containRestorationCatchmentAreaSqMi,containRestorationCatchmentAreaPercent
0,140801010101,Headwaters East Fork of the San Juan River,"{'huc12': '140801010101', 'assessmentUnitCount...",3,17.088,17.086,99.987,15.957,93.380,1.129,6.607,0.0,0.0,0.0,0.0
1,140801010102,Quartz Creek,"{'huc12': '140801010102', 'assessmentUnitCount...",3,19.675,18.866,95.887,18.648,94.779,0.735,3.736,0.0,0.0,0.0,0.0
2,140801010103,Sand Creek,"{'huc12': '140801010103', 'assessmentUnitCount...",1,18.214,18.216,100.008,18.216,100.008,0.000,0.000,0.0,0.0,0.0,0.0
3,140801010104,The Clam Shell-East Fork San Juan River,"{'huc12': '140801010104', 'assessmentUnitCount...",3,35.945,35.945,99.999,35.630,99.123,0.315,0.876,0.0,0.0,0.0,0.0
4,140801010201,Beaver Creek,"{'huc12': '140801010201', 'assessmentUnitCount...",1,16.113,16.113,99.999,16.113,99.999,0.000,0.000,0.0,0.0,0.0,0.0


## Collect Category Values

In [6]:
ircats = [];
uses   = [];
attainments = [];

for row in wbd_df2.itertuples():
    if 'summaryByIRCategory' in row.ATTAINS_RESULTS:
        for item in row.ATTAINS_RESULTS['summaryByIRCategory']:
            if item['epaIRCategoryName'] not in ircats:
                ircats.append(item['epaIRCategoryName']);
                
    if 'summaryByUse' in row.ATTAINS_RESULTS:
        for item in row.ATTAINS_RESULTS['summaryByUse']:
            if item['useName'] not in uses:
                uses.append(item['useName']);
                
            if 'useAttainmentSummary' in item:
                for smy in item['useAttainmentSummary']:
                    if smy['useAttainment'] not in attainments:
                        attainments.append(smy['useAttainment']);

ircats = sorted(ircats)
uses   = sorted(uses)
attainments = sorted(attainments)


## Generate IRCategory Columns

In [7]:
wbd_df3 = wbd_df2;

def fetch_ircat(inp,val):
    if 'summaryByIRCategory' in inp:
        for item in inp['summaryByIRCategory']:
            if item['epaIRCategoryName'] == val:
                return(
                     item['catchmentSizeSqMi']
                    ,item['catchmentSizePercent']
                    ,item['assessmentUnitCount']
                );
    return (None,None,None);
  
for ircat in ircats:
    s = wbd_df3['ATTAINS_RESULTS'].apply(fetch_ircat,args=(ircat,));
    wbd_df3['ircat_' + ircat + '_catchmentSizeSqMi']    = s.apply(lambda x: None if x is None else x[0]);
    wbd_df3['ircat_' + ircat + '_catchmentSizePercent'] = s.apply(lambda x: None if x is None else x[1]);
    wbd_df3['ircat_' + ircat + '_assessmentUnitCount']  = s.apply(lambda x: None if x is None else x[2]);

wbd_df3.tail()

,HUC12,Name,ATTAINS_RESULTS,assessmentUnitCount,totalCatchmentAreaSqMi,assessedCatchmentAreaSqMi,assessedCatchmentAreaPercent,assessedGoodCatchmentAreaSqMi,assessedGoodCatchmentAreaPercent,assessedUnknownCatchmentAreaSqMi,...,ircat_2_assessmentUnitCount,ircat_3_catchmentSizeSqMi,ircat_3_catchmentSizePercent,ircat_3_assessmentUnitCount,ircat_4A_catchmentSizeSqMi,ircat_4A_catchmentSizePercent,ircat_4A_assessmentUnitCount,ircat_5_catchmentSizeSqMi,ircat_5_catchmentSizePercent,ircat_5_assessmentUnitCount
708,140802050803,San Juan Canyon-San Juan River,"{'huc12': '140802050803', 'assessmentUnitCount...",2,38.509,31.488,81.767,0.0,0.0,29.866,...,NaN,29.866,77.555,1.0,NaN,NaN,NaN,1.622,4.212,1.0
709,140802050804,Desha Canyon Creek,"{'huc12': '140802050804', 'assessmentUnitCount...",0,19.390,0.000,0.000,0.0,0.0,0.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
710,140802050805,Deep Canyon-San Juan River,"{'huc12': '140802050805', 'assessmentUnitCount...",2,33.965,12.717,37.442,0.0,0.0,9.040,...,NaN,9.040,26.616,1.0,NaN,NaN,NaN,3.677,10.826,1.0
711,140802050806,Cha Canyon-San Juan River,"{'huc12': '140802050806', 'assessmentUnitCount...",2,43.958,22.634,51.490,0.0,0.0,19.687,...,NaN,19.687,44.786,1.0,NaN,NaN,NaN,2.947,6.704,1.0
712,140802050807,Nasia Creek-San Juan River,"{'huc12': '140802050807', 'assessmentUnitCount...",2,26.981,7.941,29.432,0.0,0.0,7.493,...,NaN,7.493,27.771,1.0,NaN,NaN,NaN,0.448,1.660,1.0


## Generate Use Columns

In [8]:
wbd_df4 = wbd_df3;

def col(val):
    val = val.replace(' ','_')
    val = val.replace('(','')
    val = val.replace(')','')
    return val;
    
def fetch_use(inp,use,attainment):
    if 'summaryByUse' in inp:
        for item in inp['summaryByUse']:
            if item['useName'] == use:
                for att in item['useAttainmentSummary']:
                    if att['useAttainment'] == attainment:
                        return (
                             att['catchmentSizeSqMi']
                            ,att['catchmentSizePercent']
                            ,att['assessmentUnitCount']
                        );
    return (None,None,None);
        
for use in uses:
    for attainment in attainments:
        s = wbd_df4['ATTAINS_RESULTS'].apply(fetch_use,args=(use,attainment));
        wbd_df4['use_' + col(use) + '_' + col(attainment) + '_catchmentSizeSqMi']    = s.apply(lambda x: None if x is None else x[0]);
        wbd_df4['use_' + col(use) + '_' + col(attainment) + '_catchmentSizePercent'] = s.apply(lambda x: None if x is None else x[1]);
        wbd_df4['use_' + col(use) + '_' + col(attainment) + '_assessmentUnitCount']  = s.apply(lambda x: None if x is None else x[2]);
        
wbd_df4.head()

,HUC12,Name,ATTAINS_RESULTS,assessmentUnitCount,totalCatchmentAreaSqMi,assessedCatchmentAreaSqMi,assessedCatchmentAreaPercent,assessedGoodCatchmentAreaSqMi,assessedGoodCatchmentAreaPercent,assessedUnknownCatchmentAreaSqMi,...,use_Wildlife_Habitat_Fully_Supporting_assessmentUnitCount,use_Wildlife_Habitat_Insufficient_Information_catchmentSizeSqMi,use_Wildlife_Habitat_Insufficient_Information_catchmentSizePercent,use_Wildlife_Habitat_Insufficient_Information_assessmentUnitCount,use_Wildlife_Habitat_Not_Assessed_catchmentSizeSqMi,use_Wildlife_Habitat_Not_Assessed_catchmentSizePercent,use_Wildlife_Habitat_Not_Assessed_assessmentUnitCount,use_Wildlife_Habitat_Not_Supporting_catchmentSizeSqMi,use_Wildlife_Habitat_Not_Supporting_catchmentSizePercent,use_Wildlife_Habitat_Not_Supporting_assessmentUnitCount
0,140801010101,Headwaters East Fork of the San Juan River,"{'huc12': '140801010101', 'assessmentUnitCount...",3,17.088,17.086,99.987,15.957,93.380,1.129,...,NaN,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN
1,140801010102,Quartz Creek,"{'huc12': '140801010102', 'assessmentUnitCount...",3,19.675,18.866,95.887,18.648,94.779,0.735,...,NaN,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN
2,140801010103,Sand Creek,"{'huc12': '140801010103', 'assessmentUnitCount...",1,18.214,18.216,100.008,18.216,100.008,0.000,...,NaN,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN
3,140801010104,The Clam Shell-East Fork San Juan River,"{'huc12': '140801010104', 'assessmentUnitCount...",3,35.945,35.945,99.999,35.630,99.123,0.315,...,NaN,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN
4,140801010201,Beaver Creek,"{'huc12': '140801010201', 'assessmentUnitCount...",1,16.113,16.113,99.999,16.113,99.999,0.000,...,NaN,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN


## Tidy up the Data Frame and Export Product

In [10]:
wbd_df5 = wbd_df4;

wbd_df5 = wbd_df5.drop(columns="ATTAINS_RESULTS");

export_excel = True;
export_geojson = False;

if export_excel:
    wbd_df5.to_excel("use_" + huc4 + ".xlsx",index = False);

if export_geojson:
    wbd_rez2 = wbd_fl.query(
        where = "SUBSTRING(HUC12,1,4) = '" + huc4 + "'"
       ,out_fields = 'HUC12'
       ,return_geometry = True
       ,orderByFields = 'HUC12'
    );
    
    exp = wbd_rez2.sdf.merge(
         right = wbd_df5
        ,how   = 'inner'
        ,on    = 'HUC12'
    );
    
    fs = FeatureSet.from_dataframe(exp);
    
    with open("use_" + huc4 + ".geojson", 'w') as f:
        f.write(fs.to_geojson);
    
